In [1]:
SEED = 0
cores = 16

from numpy.random import seed
seed(SEED)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import Counter
from xgboost import XGBClassifier
from sklearn.preprocessing import KBinsDiscretizer
from explainerdashboard import ClassifierExplainer, ExplainerDashboard

# version check
import sys
print("python version")
print(sys.version)
import sklearn
print("sklearn version = ", sklearn.__version__)
print("numpy version = ", np.__version__)
print("pandas version =", pd.__version__)

In /home/danssa/anaconda3/envs/ed/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/danssa/anaconda3/envs/ed/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/danssa/anaconda3/envs/ed/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/danssa/anaconda3/envs/ed/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/danssa/anaconda3/en

python version
3.7.10 (default, Feb 26 2021, 18:47:35) 
[GCC 7.3.0]
sklearn version =  0.24.1
numpy version =  1.20.2
pandas version = 1.2.3


In [2]:
df = pd.read_csv("~/proj_ua/data/ua_full_10types_new.v7.csv", dtype={'id':str})
df2 = df[(df['from']=="cha")]

In [3]:
#3group age split  

##step 1 finding edge value
abnormal_disc = df2.query('eGFR_ab==1').loc[:,'age']
abnormal_disc = pd.DataFrame(abnormal_disc)

est = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='kmeans')
est.fit(abnormal_disc)

ab_disc = est.transform(abnormal_disc).astype('float')
print("edges : ", est.bin_edges_[0])

age0_edge = est.bin_edges_[0][1]
age1_edge = est.bin_edges_[0][2]
age2_edge = est.bin_edges_[0][3]
print('age0_edge:', age0_edge, '\nage1_edge:', age1_edge, '\nage2_edge:', age2_edge)

abnormal_disc['level'] = abnormal_disc.apply(lambda x : 0 if x['age']<age0_edge else 1 if x['age']<age1_edge else 2, axis=1)
print('age group:\n',abnormal_disc['level'].value_counts().sort_index())

edges :  [18.         54.6101338  71.74915129 90.        ]
age0_edge: 54.61013380069844 
age1_edge: 71.74915128966373 
age2_edge: 90.0
age group:
 0    2121
1    5299
2    6334
Name: level, dtype: int64


In [4]:
##make 3group by age
df3 = df2.copy()

##age cut set up
df3['level'] = df3.apply(lambda x : 0 if x['age']<age0_edge else 1 if x['age']<age1_edge else 2, axis=1)
print(df3['level'].value_counts())

0    147679
1     46208
2     16975
Name: level, dtype: int64


In [5]:
##age0 group
X_age0 = df3[df3['level']==0]
y_age0 = X_age0['eGFR_ab'].astype("int64")

print('\nage0', X_age0['age'].min(), X_age0['age'].max())
print("total cases = %d" %X_age0.shape[0])
print("total abnormal function of kidney = %d" %sum(y_age0), "\nprevalence =", sum(y_age0)/len(y_age0)*100)

X_train0, X_test0, y_train0, y_test0 = train_test_split(X_age0, y_age0, test_size=0.2, stratify=y_age0, random_state=SEED)
print("train0 : %d" % sum(y_train0), "test0 : %d" % sum(y_test0))

##age1 group
X_age1 = df3[df3['level']==1]
y_age1 = X_age1['eGFR_ab']

print('\nage1', X_age1['age'].min(), X_age1['age'].max())
print("total cases = %d" %X_age1.shape[0])
print("total abnormal function of kidney = %d" %sum(y_age1), "\nprevalence =", sum(y_age1)/len(y_age1)*100)

X_train1, X_test1, y_train1, y_test1 = train_test_split(X_age1, y_age1, test_size=0.2, stratify=y_age1, random_state=SEED)
print("train1 : %d" % sum(y_train1), "test0 : %d" % sum(y_test1))

##age2 group
X_age2 = df3[df3['level']==2]
y_age2 = X_age2['eGFR_ab']

print('\nage2', X_age2['age'].min(), X_age2['age'].max())
print("total cases = %d" %X_age2.shape[0])
print("total abnormal function of kidney = %d" %sum(y_age2), "\nprevalence =", sum(y_age2)/len(y_age2)*100)

X_train2, X_test2, y_train2, y_test2 = train_test_split(X_age2, y_age2, test_size=0.2, stratify=y_age2, random_state=SEED)
print("train2 : %d" % sum(y_train2), "test2 : %d" % sum(y_test2))


age0 18 54
total cases = 147679
total abnormal function of kidney = 2121 
prevalence = 1.4362231596909514
train0 : 1697 test0 : 424

age1 55 71
total cases = 46208
total abnormal function of kidney = 5299 
prevalence = 11.467711218836566
train1 : 4239 test0 : 1060

age2 72 90
total cases = 16975
total abnormal function of kidney = 6334 
prevalence = 37.31369661266569
train2 : 5067 test2 : 1267


In [6]:
##concat both trainset and testset
X_train = pd.concat([X_train0, X_train1, X_train2])
y_train = pd.concat([y_train0, y_train1, y_train2])

X_test = pd.concat([X_test0, X_test1, X_test2])
y_test = pd.concat([y_test0, y_test1, y_test2])

print("total cases = %d" % (X_train.shape[0] + X_test.shape[0]))
print("total abnormal function of kidney = %d" % (sum(y_train) + sum(y_test)), "\nprevalence =", (sum(y_train)+sum(y_test))/(len(y_train)+len(y_test))*100)

total cases = 210862
total abnormal function of kidney = 13754 
prevalence = 6.52274947596058


In [7]:
X_train_features = X_train.loc[:, ['male', 'he_usg', 'he_uph', 'he_ubld', 'he_uglu', 'he_upro', 'age']]

print('%d train cases, %d variables' % (X_train_features.shape[0], X_train_features.shape[1]))
print('%d test cases'%X_test.shape[0])

168689 train cases, 7 variables
42173 test cases


In [8]:
#standardization

scaler = StandardScaler()
std_cols=['age','he_uph','he_usg']
std_df=X_train_features[std_cols]

X_train_features[std_cols]=scaler.fit_transform(std_df)
X_train_features.describe()

,male,he_usg,he_uph,he_ubld,he_uglu,he_upro,age
count,168689.000000,1.686890e+05,1.686890e+05,168689.000000,168689.000000,168689.000000,1.686890e+05
mean,0.421658,4.491159e-15,4.892829e-16,0.774259,0.197031,0.327241,8.626475e-17
std,0.493826,1.000003e+00,1.000003e+00,1.264018,0.819329,0.764392,1.000003e+00
min,0.000000,-1.722714e+00,-1.468296e+00,0.000000,0.000000,0.000000,-1.854108e+00
25%,0.000000,-1.090873e+00,-8.579901e-01,0.000000,0.000000,0.000000,-7.562719e-01
50%,0.000000,1.728108e-01,-2.476840e-01,0.000000,0.000000,0.000000,-1.750646e-01
75%,1.000000,8.046524e-01,9.729283e-01,1.000000,0.000000,0.000000,7.290358e-01
max,1.000000,1.436494e+00,3.414153e+00,5.000000,5.000000,5.000000,2.795551e+00


In [9]:
#testset
X_test_features = X_test.loc[:, ['male', 'he_usg', 'he_uph', 'he_ubld', 'he_uglu', 'he_upro', 'age']]
std_df_test=X_test_features[std_cols]
X_test_features[std_cols]=scaler.transform(std_df_test)

X_test_features.describe()

,male,he_usg,he_uph,he_ubld,he_uglu,he_upro,age
count,42173.000000,42173.000000,42173.000000,42173.000000,42173.00000,42173.000000,42173.000000
mean,0.422688,0.000891,0.001935,0.767434,0.20110,0.324924,-0.003618
std,0.493992,0.999762,0.996091,1.262301,0.82853,0.761808,1.005477
min,0.000000,-1.722714,-1.468296,0.000000,0.00000,0.000000,-1.854108
25%,0.000000,-1.090873,-0.857990,0.000000,0.00000,0.000000,-0.756272
50%,0.000000,0.172811,-0.247684,0.000000,0.00000,0.000000,-0.175065
75%,1.000000,0.804652,0.972928,1.000000,0.00000,0.000000,0.729036
max,1.000000,1.436494,3.414153,4.000000,5.00000,5.000000,2.795551


In [10]:
#model : ua_cha60_10types.v5
model = XGBClassifier(colsample_bytree=0.6, gamma=0.8, learning_rate=0.01, max_depth=3, 
                          min_child_weight=6, n_estimators=500, objective="binary:logistic",
                          reg_alpha=0.5, reg_lambda=2, scale_pos_weight=14, subsample=0.2, 
                          nthread=6, random_state=SEED).fit(X_train_features, y_train)

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[15:02:33] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [11]:
explainer = ClassifierExplainer(model, X_test_features, y_test, 
                               shap='tree',
                               model_output='probability')

Detected XGBClassifier model: Changing class type to XGBClassifierExplainer...
Note: model_output=='probability'. For XGBClassifier shap values normally get calculated against X_background, but paramater X_background=None, so using X instead
Generating self.shap_explainer = shap.TreeExplainer(model, X, model_output='probability', feature_perturbation='interventional')...
Note: Shap interaction values will not be available. If shap values in probability space are not necessary you can pass model_output='logodds' to get shap values in logodds without the need for a background dataset and also working shap interaction values...


In [ ]:
ExplainerDashboard(explainer,
                   importances=True,
                   model_summary=True,
                   contributions=True,
                   whatif=True,
                   shap_dependence=True,
                   shap_interaction=True,
                   decision_trees=True
                  ).run()

Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
For this type of model and model_output interactions don't work, so setting shap_interaction=False...
Generating layout...
Calculating shap values...


100%|===================| 42061/42173 [03:31<00:00]        

Generating xgboost model dump...
Calculating dependencies...
Calculating permutation importances (if slow, try setting n_jobs parameter)...
Calculating predictions...
Calculating liftcurve_dfs...
Calculating prediction probabilities...
Calculating classification_dfs...
Calculating confusion matrices...
Calculating pr auc curves...
Calculating pred_percentiles...
Calculating roc auc curves...
Calculating metrics...
Calculating ShadowDecTree for each individual decision tree...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Starting ExplainerDashboard on http://192.168.0.223:8050
Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8050/ (Press CTRL+C to quit)
221.150.239.1 - - [15/Oct/2021 15:06:58] "GET / HTTP/1.1" 200 -
221.150.239.1 - - [15/Oct/2021 15:06:58] "GET /assets/bootstrap.min.css?m=1616526662.0 HTTP/1.1" 200 -
221.150.239.1 - - [15/Oct/2021 15:06:58] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_1m1617985068.14.0.min.js HTTP/1.1" 200 -
221.150.239.1 - - [15/Oct/2021 15:06:58] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_1m1617985068.7.2.min.js HTTP/1.1" 200 -
221.150.239.1 - - [15/Oct/2021 15:06:58] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_1m1617985068.8.7.min.js HTTP/1.1" 200 -
221.150.239.1 - - [15/Oct/2021 15:06:58] "GET /_dash-component-suites/dash_table/bundle.v4_11_3m1617985106.js HTTP/1.1" 200 -
221.150.239.1 - - [15/Oct/2021 15:06:58] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_9_1m1617985068.14.0.min.js HTTP/1.1" 200 -
221.150.239.1 - - [15/Oct/2021 15:06:58] "GET /_dash-component-suites/dash_html_comp